# Exploratory Data Analysis

In [1]:
import os 
#from utils.utils import *

from fog.code.utils.utils import *
from pyspark.sql.functions import lit, col, sum
from pyspark.sql.types import StringType, BooleanType, IntegerType, FloatType, DateType
spark = SparkSession.builder.appName("PySpark Cloud Test").getOrCreate()
# conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '16g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','16g')])

In [2]:
fog_path = "parkinsons_data/train/processed/defog_tasks"
fog_files = list_blobs("msca-bdp-student-gcs", string_match=fog_path)

len(fog_files)

13

In [ ]:
# df = feed_files(fog_files, prefix=fog_path, spark_session=spark, file_type="parquet")

In [3]:
df = spark.read.parquet("gs://msca-bdp-student-gcs/parkinsons_data/train/processed/defog_tasks")

In [4]:
df.printSchema()

root
 |-- Subject: string (nullable = true)
 |-- Visit: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- AccV: float (nullable = true)
 |-- AccML: float (nullable = true)
 |-- AccAP: float (nullable = true)
 |-- StartHesitation: integer (nullable = true)
 |-- Turn: integer (nullable = true)
 |-- Walking: integer (nullable = true)
 |-- Valid: boolean (nullable = true)
 |-- Task: boolean (nullable = true)
 |-- SourceDefog: integer (nullable = true)
 |-- Medication: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- YearsSinceDx: integer (nullable = true)
 |-- UPDRSIII_On: integer (nullable = true)
 |-- UPDRSIII_Off: integer (nullable = true)
 |-- NFOGQ: integer (nullable = true)
 |-- Test: string (nullable = true)
 |-- TimeSeconds: double (nullable = true)
 |-- Begin: double (nullable = true)
 |-- End: double (nullable = true)
 |-- TaskType: string (nullable = true)
 |-- MB9: integer (nul

In [5]:
df.count()

13525703

In [6]:
df.select("target").groupBy("target").count().show()

+------+--------+
|target|   count|
+------+--------+
|     1|     500|
|     3|   92401|
|     2|  568265|
|     0|12864537|
+------+--------+



In [ ]:
df.select("MB9", "Rest1", "MB7").show(5)

In [ ]:
df1.select(sum(df1.StartHesitation),sum(df1.Turn),sum(df1.Walking),sum(df1.target)).show()

In [ ]:
df.select(sum(df.StartHesitation),sum(df.Turn),sum(df.Walking)).show()

In [ ]:
# does sum of target match sum of each target variable?
(1000 * 1) + (1136530 * 2) + (184802 * 3) == 2828466